In [ ]:
!pip -q install sentence-transformers faiss-cpu huggingface-hub python-dotenv

import os
import re
import pickle
import faiss
import numpy as np
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')
assert HF_TOKEN, "HF_TOKEN not found in Colab secrets! Please add it first."

GENERATION_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

client = InferenceClient(model=GENERATION_MODEL, token=HF_TOKEN)

TXT_PATH = "/content/master_dataset_v4.txt"

with open(TXT_PATH, "r", encoding="utf-8", errors="ignore") as f:
    raw_text = f.read()

def paragraph_split(text: str) -> List[str]:
    paras = re.split(r"\n\s*\n", text.strip())
    paras = [re.sub(r"[ \t]+", " ", p.strip()) for p in paras if p.strip()]
    return paras

def make_chunks(paras: List[str], max_chars=1400, overlap=180) -> List[Dict]:
    chunks, buf, size = [], [], 0
    for p in paras:
        if len(p) > max_chars:
            for i in range(0, len(p), max_chars):
                seg = p[i:i + max_chars]
                chunks.append({"text": seg})
            continue
        if size + len(p) + 1 <= max_chars:
            buf.append(p)
            size += len(p) + 1
        else:
            if buf:
                chunk = " ".join(buf)
                chunks.append({"text": chunk})
                tail = chunk[-overlap:]
                buf = [tail, p]
                size = len(tail) + 1 + len(p)
            else:
                chunks.append({"text": p})
                buf, size = [], 0
    if buf:
        chunks.append({"text": " ".join(buf)})
    for i, c in enumerate(chunks):
        c["id"] = f"chunk-{i:05d}"
    return chunks

paras = paragraph_split(raw_text)
chunks = make_chunks(paras)

EMB_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL)

corpus_texts = [c["text"] for c in chunks]
emb = embedder.encode(
    corpus_texts,
    batch_size=128,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb)

faiss.write_index(index, "/content/faiss.index")
with open("/content/chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

def retrieve(query: str, k=5):
    q = embedder.encode([query], normalize_embeddings=True)
    D, I = index.search(q, k)
    results = []
    for score, idx in zip(D[0].tolist(), I[0].tolist()):
        if idx == -1:
            continue
        results.append({
            "id": chunks[idx]["id"],
            "score": round(float(score), 4),
            "text": chunks[idx]["text"]
        })
    return results

SYSTEM = """You are AlgoTutor, an algorithms & data-structures tutor.
Answer USING ONLY the provided context passages.
If the answer is not in the context, say you don't know.
Cite the chunk IDs you used in the form [ID]. Keep answers concise but clear.
"""

def build_prompt(query: str, passages: list) -> str:
    ctx = ""
    for p in passages:
        ctx += f"[{p['id']}] {p['text']}\n\n"
    prompt = (
        f"{SYSTEM}\n\n"
        f"Question: {query}\n\n"
        f"Context Passages:\n{ctx}\n"
        f"Answer:"
    )
    return prompt

def rag_answer(query: str, k=5, model=GENERATION_MODEL, max_tokens=600):
    topk = retrieve(query, k=k)
    prompt = build_prompt(query, topk)

    response = client.chat_completion(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7,
    )

    answer = response.choices[0].message["content"].strip()
    return answer, topk


ans, ctx = rag_answer("Explain amortized analysis of dynamic arrays and show the potential method.")
print("\nAnswer:\n", ans)

ans, ctx = rag_answer("Explain the concept of data structure?")
print("\nAnswer:\n", ans)


Total chunks: 2735


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

FAISS index built successfully!
[{'id': 'chunk-00880', 'score': 0.6292, 'text': 'er to use Θ-notation in the running time equation and give the asymptotic tight bound like in the selection sort. For algorithm such as insertion sort, whose complexity varies as the input data distribution we conventionally analyze its worst-case and use O-notation.\n192 10. ALGORITHM COMPLEXITY ANALYSIS\n10.5 *Amortized Analysis\nThere are two different ways to evaluate an algorithm/data structure:\n1. Consider each operation separately: one that look each operation incurred in the algorithm/data structure separately and offers worst-case running time O and average running time Θ for each operation. For the whole algorithm, it sums up on these two cases by how many times each operation is incurred.\n2. Amortized among a sequence of (related) operations: Amortized analysis can be used to show that the average cost of an operation is small, if one averages over a sequence of operations, even though a simpl

In [ ]:
ans, ctx = rag_answer("What is the key property of a binary heap?")
print("\nAnswer:\n", ans)

ans, ctx = rag_answer("Explain the time complexity of Quicksort.")
print("\nAnswer:\n", ans)

ans, ctx = rag_answer("Write a short pseudocode for bubble sort.")
print("\nAnswer:\n", ans)


Answer:
 The key property of a binary heap is that it satisfies the heap ordering property, specifically, the value of each node is less than or equal to the value of its parents [chunk-00230]. Therefore, the maximum value is found at the root. It is a tree based data structure with at most two children per node, and is commonly used to implement a priority queue where the highest priority item can be accessed in O(log n) time.

Answer:
 The time complexity of Quicksort is analyzed using the recurrence relation T(n) = T(n/2) + O(n) [ID: 15.6], which with the Master Theorem gives us T(n) = O(n) [ID: 15.7]. Quicksort's efficiency comes from its divide-and-conquer approach and its average-case time complexity, which is O(n log n). However, the worst-case time complexity can be O(n^2) when the pivot fails to split the array evenly. Optimizations to Quicksort include using a better pivot selection algorithm like the "median of three" [ID: last paragraph].

Answer:
 Pseudocode for bubble so

In [ ]:
ans, ctx = rag_answer("What is a pointer in the context of data structures?")
print("\nAnswer:\n", ans)


Answer:
 A pointer is a variable that holds the address of an item in the context of data structures, particularly linked lists. It is used to link together different items in the list [chunk-00716, 00136]. The linked list is a dynamic data structure where each node contains a value and a pointer to the next node [chunk-00716, 00136]. The pointer allows traversing through the list and adding or removing nodes at any position [chunk-00716].


In [ ]:
ans, ctx = rag_answer("What is the main difference between a queue and a stack?")
print("\nAnswer:\n", ans)


Answer:
 The main difference between a queue and a stack is the order in which elements are removed. In a stack, elements are removed in the order they were last added (last-in-first-out or LIFO), while in a queue, elements are removed in the order they were first added (first-in-first-out or FIFO). [ID: 1.1, 1.2.2, 1.3, 1.4]
